In [8]:
from env import get_connection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from math import sqrt
from scipy import stats
from pydataset import data
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import acquire
import prepare


# Here we acquire the data from SQL using a function 
   - We only pulled down yearbuilt, taxvaluedollarcnt, taxamount, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet from the 2017 property table 

In [17]:
import os

def get_zillow():
    filename = "titanic.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql('SELECT yearbuilt, taxvaluedollarcnt, taxamount, bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet ,fips FROM properties_2017 JOIN propertylandusetype USING(propertylandusetypeid) WHERE propertylandusetypeid = 261; ', get_connection('zillow'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df 

In [18]:
df= get_zillow()

In [23]:
df.head(20)

,yearbuilt,taxvaluedollarcnt,taxamount,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,fips
4,2005.0,296425.0,6941.39,4.0,2.0,3633.0,6037.0
6,2011.0,847770.0,10244.94,3.0,4.0,1620.0,6037.0
7,1926.0,646760.0,7924.68,3.0,2.0,2077.0,6037.0
11,1972.0,5328.0,91.60,0.0,0.0,1200.0,6037.0
14,1973.0,6920.0,255.17,0.0,0.0,171.0,6037.0
15,1960.0,14166.0,163.79,0.0,0.0,203.0,6037.0
18,1950.0,169471.0,2532.88,3.0,1.0,1244.0,6037.0
19,1950.0,233266.0,3110.99,3.0,2.0,1300.0,6037.0
20,1951.0,290492.0,3870.25,3.0,2.0,1222.0,6037.0
21,2016.0,1303522.0,14820.10,4.0,4.0,4144.0,6037.0


# Prepare
  - Now we will get our data ready for future use
      - Checking and getting rid of nulls
      - Checking if data types are correct
      - Splitting our data into Train, Validate, Test

In [20]:
df.isnull().sum()

yearbuilt                       9337
taxvaluedollarcnt                493
taxamount                       4442
bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
fips                               0
dtype: int64

In [28]:
df.rename(columns= {'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'sqft', 
                    'yearbuilt': 'year_built',
                    'taxamount': 'tax_amount', 'taxvaluedollarcnt': 'tax_value'}, inplace= True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   year_built  float64
 1   tax_value   float64
 2   tax_amount  float64
 3   bedrooms    float64
 4   bathrooms   float64
 5   sqft        float64
 6   fips        float64
dtypes: float64(7)
memory usage: 130.6 MB


In [30]:
df.isnull().sum()

year_built    0
tax_value     0
tax_amount    0
bedrooms      0
bathrooms     0
sqft          0
fips          0
dtype: int64

In [33]:
df= df.dropna()
df.isnull().sum()

year_built    0
tax_value     0
tax_amount    0
bedrooms      0
bathrooms     0
sqft          0
fips          0
dtype: int64

In [ ]:
plt.figure(figsize= (16, 6))

cols = ['year_built', 'tax_value', 'tax_amount', 'bedrooms', 'bathrooms', 'sqft']